In [30]:
path = '/Users/z.abidin/CC-NEWS-20220228233520-00697.warc.gz'

In [52]:
%%python - --input_base_url /Users/z.abidin/dev/0/commoncrawl/cc-pyspark/data/ --num_output_partitions 8 --log_level WARN ./warc.txt output7
import ujson as json
try:
    # Python2
    from urlparse import urlparse
except ImportError:
    # Python3
    from urllib.parse import urlparse
from pyspark.sql.types import StructType, StructField, StringType, LongType
from sparkcc import CCSparkJob
from bs4 import BeautifulSoup
import shutil
import re

def extract_generator(x):
    try:
        meta = x.findAll('meta')
        return list(filter(lambda x: x.get('name') == 'generator', meta))[0].get('content')
    except Exception as e:
        return None
    
email_tester = re.compile("[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+")
def extract_all_emails(x):
    try:
        return '|'.join(set(email_tester.findall(x)))
    except Exception as e:
        return None

with open('contact_keywords.txt', 'r') as f:
    contact_kw = f.read().splitlines()

def contact_filter_by_kw(x):
    for kw in contact_kw:
        flag = False
        try:
            flag = x.get('href').lower().find(kw) >=0
        except Exception as e:
            return e
        if flag: return True
        try:
            flag = x.text.lower().find(kw) >=0
        except Exception as e:
            return e
        if flag: return True

    return flag
def extract_ssl_or_not(x):
    try:
        return urlparse(x).scheme
    except Exception as e:
        return None
    
def extract_contact(x):
    try:
        aa = x.findAll('a')
        return '|'.join(set(map(lambda x: x.get('href'), filter(contact_filter_by_kw, aa))))
    except Exception as e:
        return None

def extract_country_lang(x):
    try:
        lang_country = x.find('html').get('lang')
        a = lang_country.split('-')
        if len(a) == 2:
            lang, country = a
        elif len(a) == 1:
            lang, country = a[0], None 
        else:
            lang, country = None, None    
        return lang, country
    except Exception as e:
        return None, None
    
class Job(CCSparkJob):
    output_schema = StructType([
        StructField('conent_type', StringType(), True),
        StructField('record_type', StringType(), True),
        StructField('uri', StringType(), True),
        StructField('is_secure', StringType(), True),
        StructField('language', StringType(), True),
        StructField('country', StringType(), True),
        StructField('generator', StringType(), True),
        StructField('emails', StringType(), True),
        StructField('contact', StringType(), True)
    ])
    
    def process_record(self, record):
        record_type = record.rec_type
        uri = record.rec_headers.get_header('WARC-Target-URI')

        if record_type == 'response' and record.http_headers.get_header('Content-Type').find('text/html') >= 0:
            try:
                content = str(
                    record\
                    .content_stream()\
                    .read()\
                    .decode('utf-8')
                )
                bs = BeautifulSoup(content, features="lxml")
                generator = extract_generator(bs)
                contact = extract_contact(bs)
                lang, country = extract_country_lang(bs)
                emails  = extract_all_emails(content)
                is_secure = extract_ssl_or_not(uri)
                yield ('text/html', record_type, uri, is_secure, lang, country, generator, emails, contact)
            except Exception as e:
                yield ('error', record_type, uri, None, None, None, None, None, None)
            
if __name__ == "__main__":
    job = Job()
    job.run()


22/12/04 17:30:31 WARN Utils: Your hostname, C02GJ03FML88.local resolves to a loopback address: 127.0.0.1; using 192.168.8.105 instead (on interface en0)
22/12/04 17:30:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 17:30:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/04 17:30:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Traceback (most recent call last):
  File "/Users/z.abidin/opt/anaconda3/envs/cc-pyspark/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/z.abidin/opt/anaconda3/envs/cc-pyspark/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/z.abidin/opt/anaconda3/envs/cc-pyspark/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/z.abidin/opt/anaconda3/envs/cc-pyspark/lib/python3.10/site-packages/pyspark/context.py", line 197, in __init__
    self._do_init(
  File "/Users/z.abidin/opt/anaconda3/envs/cc-pyspark/lib/python3.10/site-packages/pyspark/context.py", line 349, in _do_init
    self._temp_dir = self._jvm.org.apache.spark.util.Utils.create

Error while terminating subprocess (pid=21374): 


In [45]:
from pyspark.sql import SparkSession
sess = SparkSession.builder.getOrCreate()

In [50]:
df = sess.read.parquet('spark-warehouse/output7')

In [51]:
df.show()

+-----------+-----------+--------------------+---------+--------+-------+--------------------+--------------------+--------------------+
|conent_type|record_type|                 uri|is_secure|language|country|           generator|              emails|             contact|
+-----------+-----------+--------------------+---------+--------+-------+--------------------+--------------------+--------------------+
|  text/html|   response|https://www.hindu...|    https|      en|   null|net.razorvine.pic...|                    |net.razorvine.pic...|
|  text/html|   response|https://rotativo....|    https|      es|     MX|AMP for WP 1.0.77.36|                    |net.razorvine.pic...|
|  text/html|   response|https://copainsda...|    https|      fr|   null|net.razorvine.pic...|                    |net.razorvine.pic...|
|  text/html|   response|https://lajmi.net...|    https|      en|     US|     WordPress 5.2.1|                    |net.razorvine.pic...|
|  text/html|   response|https://diariodo